# US CRIME ANALYSIS

# PHASE 1: ENVIRONMENT SETUP

In [15]:
import requests
import time as t
import pandas as pd
import datetime
import pprint
from datetime import date 
from bs4 import BeautifulSoup

import glob
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as po

import statistics as stats
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# from collections import Counter
# from string import punctuation

In [16]:
program_start = t.time()

#  PHASE 2: ETL

In [17]:
# import listings-2.csv into pandas dataframe
data_raw = pd.read_csv("data/crime_data.csv", index_col=0) 
data_raw.head(5)

,X,REPORT_DAT,SHIFT,OFFENSE,METHOD,BLOCK,DISTRICT,PSA,WARD,ANC,...,year,month,day,hour,minute,second,EW,NS,quad,crimetype
1,1,8/31/2008 8:47:00 PM,EVENING,THEFT/OTHER,OTHERS,3500 - 3599 BLOCK OF R STREET NW,2.0,206.0,2,2E,...,2008,8,31,20,47,0,West,North,Northwest,Non-Violent
2,2,9/1/2008 12:45:00 AM,MIDNIGHT,MOTOR VEHICLE THEFT,OTHERS,2000 - 2015 BLOCK OF O STREET NW,2.0,208.0,2,2B,...,2008,9,1,0,45,0,West,North,Northwest,Non-Violent
3,3,9/1/2008 3:00:00 AM,MIDNIGHT,THEFT/OTHER,OTHERS,700 - 799 BLOCK OF 11TH STREET NW,1.0,101.0,2,2C,...,2008,9,1,3,0,0,East,North,Northeast,Non-Violent
4,4,9/9/2008 7:46:00 AM,DAY,THEFT/OTHER,OTHERS,1700 - 1799 BLOCK OF P STREET NW,2.0,208.0,2,2B,...,2008,9,9,7,46,0,West,North,Northwest,Non-Violent
5,5,8/24/2008 8:00:00 PM,EVENING,MOTOR VEHICLE THEFT,OTHERS,500 - 599 BLOCK OF INDIANA AVENUE NW,1.0,102.0,2,2C,...,2008,8,24,20,0,0,East,North,Northeast,Non-Violent


In [18]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342867 entries, 1 to 342867
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   X                     342867 non-null  int64  
 1   REPORT_DAT            342867 non-null  object 
 2   SHIFT                 342867 non-null  object 
 3   OFFENSE               342867 non-null  object 
 4   METHOD                342867 non-null  object 
 5   BLOCK                 342867 non-null  object 
 6   DISTRICT              342667 non-null  float64
 7   PSA                   342616 non-null  float64
 8   WARD                  342867 non-null  int64  
 9   ANC                   342867 non-null  object 
 10  NEIGHBORHOOD_CLUSTER  338162 non-null  object 
 11  BLOCK_GROUP           341776 non-null  object 
 12  CENSUS_TRACT          341776 non-null  float64
 13  VOTING_PRECINCT       342783 non-null  object 
 14  CCN                   342867 non-null  int64  
 15  

In [19]:
data_raw.describe()

,X,DISTRICT,PSA,WARD,CENSUS_TRACT,CCN,XBLOCK,YBLOCK,year,month,day,hour,minute,second
count,342867.000000,342667.000000,342616.000000,342867.000000,341776.000000,3.428670e+05,342867.000000,342867.000000,342867.000000,342867.000000,342867.000000,342867.000000,342867.000000,342867.000000
mean,171434.000000,3.727415,378.080069,4.449501,6218.780511,1.265285e+07,-77.008139,38.905970,2012.543651,6.655686,15.979505,13.228853,26.294750,6.789986
std,98977.321711,1.977502,197.334627,2.378859,3137.022064,2.845777e+06,0.036324,0.031595,2.810362,3.327466,8.760185,6.233693,18.106904,14.926954
min,1.000000,1.000000,101.000000,1.000000,100.000000,1.000600e+05,-77.113642,38.813470,2008.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,85717.500000,2.000000,206.000000,2.000000,3500.000000,1.012491e+07,-77.032380,38.890940,2010.000000,4.000000,8.000000,9.000000,10.000000,0.000000
50%,171434.000000,4.000000,401.000000,5.000000,7000.000000,1.303074e+07,-77.013122,38.906430,2013.000000,7.000000,16.000000,14.000000,28.000000,0.000000
75%,257150.500000,5.000000,507.000000,6.000000,8904.000000,1.510258e+07,-76.985523,38.925286,2015.000000,9.000000,23.000000,18.000000,42.000000,0.000000
max,342867.000000,7.000000,708.000000,8.000000,11100.000000,9.943899e+07,-76.910010,38.994901,2017.000000,12.000000,31.000000,23.000000,59.000000,59.000000


In [20]:
col_list = list(data_raw.columns)
col_list

['X',
 'REPORT_DAT',
 'SHIFT',
 'OFFENSE',
 'METHOD',
 'BLOCK',
 'DISTRICT',
 'PSA',
 'WARD',
 'ANC',
 'NEIGHBORHOOD_CLUSTER',
 'BLOCK_GROUP',
 'CENSUS_TRACT',
 'VOTING_PRECINCT',
 'CCN',
 'START_DATE',
 'END_DATE',
 'XBLOCK',
 'YBLOCK',
 'optional',
 'date',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'EW',
 'NS',
 'quad',
 'crimetype']

In [10]:
col_list_length = len(col_list)
col_list_length

31